# 07 Multiple Regression, Dummy-Codierung & Panelmodelle

In diesem Notebook erweitern wir die bisherige Regressionsanalyse um Themen aus Vorlesung 11 – Multiple Regression & Panelmodelle. Gemäß den Lernzielen der Vorlesung sollen wir
- multiple lineare Modelle formulieren und interpretieren,
- Dummy-Variablen korrekt erstellen und deuten,
- Multikollinearität diagnostizieren (mittels `Variance Inflation Factor`), und
- einen ersten Einblick in Panelmodelle gewinnen.

Wir nutzen weiterhin den OECD-Datensatz aus Notebook 05. Für die Regressionsmodelle benötigen wir eine Beobachtung pro Land (Pseudoreplikation vermeiden), weshalb wir den **Snapshot-Datensatz** `oecd_snapshot_latest.csv` verwenden. Für das Panelmodell (Fixed Effects) greifen wir auf den **Zeitreihendatensatz** `oecd_full_time_series.csv` zurück.

*Hinweis:* Die folgenden Codezellen laden die Daten aus dem Repository und führen die Berechnungen aus. Falls Sie dieses Notebook außerhalb des Projekts ausführen, passen Sie die Dateipfade entsprechend an.

## 1. Daten laden und vorbereiten

Wir importieren zunächst die benötigten Pakete. Anschließend laden wir den Snapshot-Datensatz und bereiten ihn analog zu Notebook 05 vor. Dabei fokussieren wir uns auf die Variablen *FeelingSafe* (abhängige Variable), *Homicides*, *SocialSupport* und *LifeSat*. Weitere potenzielle Einflussfaktoren können je nach Analyse ergänzt werden.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
from pathlib import Path

# Ein schöneres Plot-Design verwenden
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Dateipfade definieren
base_dir = Path.cwd()
snapshot_file = 'oecd_snapshot_latest.csv'
timeseries_file = 'oecd_full_time_series.csv'

# Versuchen, die Dateien im lokalen Projekt zu finden
candidate_dirs = [base_dir/'data', base_dir.parent/'data', Path('data')]
snapshot_path = None
timeseries_path = None
for d in candidate_dirs:
    if snapshot_path is None and (d / snapshot_file).exists():
        snapshot_path = d / snapshot_file
    if timeseries_path is None and (d / timeseries_file).exists():
        timeseries_path = d / timeseries_file

if snapshot_path is None:
    raise FileNotFoundError(f'Konnte {snapshot_file} nicht finden. Bitte Pfad anpassen.')
if timeseries_path is None:
    raise FileNotFoundError(f'Konnte {timeseries_file} nicht finden. Bitte Pfad anpassen.')

# Daten laden
df_snap = pd.read_csv(snapshot_path)
df_ts = pd.read_csv(timeseries_path)

# Daten kurz inspizieren
display(df_snap.head())
display(df_ts.head())


### 1.1 Snapshot-Datensatz aufbereiten

Wir filtern auf die relevanten Kennzahlen und erzeugen anschließend ein breites Format mit je einer Zeile pro Land. Damit erhalten wir die Matrix der Prädiktoren. Um starke Schiefe zu korrigieren, bilden wir den Logarithmus der Mordrate.

In [ ]:

# Variablen von Interesse
measures = ['Feeling safe at night', 'Homicides', 'Social support', 'Life satisfaction']

df_filtered = df_snap[df_snap['measure'].isin(measures)].copy()
# Numerische Werte erzwingen
df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')
# Pivotieren: eine Zeile pro Land, Spalten nach Kennzahl
df_pivot = df_filtered.pivot_table(index='reference_area', columns='measure', values='value', aggfunc='mean')
# Zeilen mit fehlenden Werten entfernen
df_pivot = df_pivot.dropna()
# Spaltennamen vereinfachen
df_pivot = df_pivot.rename(columns={
    'Feeling safe at night': 'FeelingSafe',
    'Homicides': 'Homicides',
    'Social support': 'SocialSupport',
    'Life satisfaction': 'LifeSat'
})

# Logarithmus der Mordrate bilden (kleine Konstante gegen log(0))
df_pivot['Log_Homicides'] = np.log(df_pivot['Homicides'] + 1e-6)

print(f'Anzahl Länder im Datensatz: {len(df_pivot)}')
display(df_pivot.head())


## 2. Multiple Regression

Die Multiple Regression ermöglicht es, den isolierten Effekt jeder erklärenden Variable zu schätzen, während alle anderen Variablen konstant gehalten werden (ceteris-paribus-Interpretation). Wie in der Vorlesung erläutert, trennt sie Einflussfaktoren voneinander und erlaubt realistischere Modelle als die einfache Regression.

In [ ]:

# Regressionsdesign
Y = df_pivot['FeelingSafe']
X = df_pivot[['Log_Homicides', 'SocialSupport', 'LifeSat']]
# Konstante hinzufügen
X = sm.add_constant(X)

# Modell schätzen
model_multi = sm.OLS(Y, X).fit()
print(model_multi.summary())


#### Interpretation

Die obige Tabelle zeigt Koeffizienten, Standardfehler, t-Werte und p-Werte des multiplen Modells. Wichtig ist die Interpretation der Steigungsparameter:

- Der Koeffizient für `Log_Homicides` misst, wie stark das Sicherheitsgefühl sinkt, wenn die logarithmierte Mordrate um eine Einheit steigt, **bei konstanter SocialSupport und LifeSat**.
- `SocialSupport` zeigt, wie stark die Unterstützung das Sicherheitsgefühl erhöht, wenn Mordrate und Lebenszufriedenheit konstant gehalten werden.
- `LifeSat` spiegelt den Zusammenhang zwischen allgemeiner Lebenszufriedenheit und Sicherheitsgefühl wider.

Ein signifikantes F-Statistik (p < 0,05) und ein relativ hoher Adjusted R² deuten darauf hin, dass das Modell einen substantiellen Anteil der Varianz erklärt.

## 3. Dummy-Variablen

Um den Umgang mit kategorialen Einflussgrößen zu demonstrieren, erstellen wir eine Dummy-Variable. Da der Snapshot-Datensatz keine offensichtliche Kategorisierung wie Geschlecht oder Bildungsgrad enthält (nach dem Pivotieren liegt eine Zeile pro Land vor), klassifizieren wir Länder anhand ihrer sozialen Unterstützung: Länder mit überdurchschnittlicher SocialSupport erhalten die Dummy-Variable `HighSocial` = 1, alle anderen 0.

In [ ]:

# Dummy-Variable HighSocial: 1, wenn SocialSupport oberhalb des Medians liegt
median_support = df_pivot['SocialSupport'].median()
df_pivot['HighSocial'] = (df_pivot['SocialSupport'] > median_support).astype(int)

# Neues Designmatrix mit Dummy
X_dummy = df_pivot[['Log_Homicides', 'HighSocial', 'LifeSat']]
X_dummy = sm.add_constant(X_dummy)

model_dummy = sm.OLS(Y, X_dummy).fit()
print(model_dummy.summary())


#### Interpretation der Dummy-Variable

Die Dummy-Variable `HighSocial` repräsentiert den Sprung im Sicherheitsgefühl zwischen Ländern mit hoher und niedriger sozialer Unterstützung. Ein positiver und signifikanter Koeffizient deutet darauf hin, dass sich Menschen in Ländern mit überdurchschnittlicher sozialer Unterstützung tendenziell sicherer fühlen als in Ländern mit unterdurchschnittlicher Unterstützung, während Mordrate und Lebenszufriedenheit kontrolliert werden.

## 4. Multikollinearität diagnostizieren

Mehrere erklärende Variablen können untereinander stark korreliert sein, was zu instabilen Koeffizientenschätzungen führt. In der Vorlesung wurde dazu der **Variance Inflation Factor (VIF)** vorgestellt. Für jede Variable berechnen wir den VIF – Werte über 5–10 gelten als Hinweis auf problematische Multikollinearität.

In [ ]:

# VIF-Berechnung für das multiple Modell
vif_data = pd.DataFrame()
vif_data['Variable'] = ['Log_Homicides', 'SocialSupport', 'LifeSat']
vif_data['VIF'] = [variance_inflation_factor(df_pivot[['Log_Homicides', 'SocialSupport', 'LifeSat']].values, i)
                    for i in range(3)]
vif_data


#### Interpretation

Wenn die VIF-Werte unter 5 liegen, besteht keine ernsthafte Multikollinearität. Höhere Werte legen nahe, dass einzelne Prädiktoren stark mit anderen korrelieren. In diesem Fall sollte geprüft werden, ob alle Variablen benötigt werden oder ob bestimmte Variablen kombiniert oder entfernt werden sollten.

## 5. Konfundierung (Confounding)

Konfundierung beschreibt das Phänomen, dass der Effekt einer Variable auf die Zielgröße durch andere Variablen verfälscht wird. Im Beispiel aus Notebook 05 führte die Berücksichtigung von `SocialSupport` dazu, dass der Effekt der Mordrate geringer wurde. Wir demonstrieren diesen Effekt, indem wir das Modell mit und ohne SocialSupport schätzen und die Änderung des Koeffizienten für `Log_Homicides` vergleichen.

In [ ]:

# Modell ohne SocialSupport
X_simple = sm.add_constant(df_pivot[['Log_Homicides']])
model_simple = sm.OLS(Y, X_simple).fit()

# Modell mit SocialSupport
X_with_support = sm.add_constant(df_pivot[['Log_Homicides', 'SocialSupport']])
model_with_support = sm.OLS(Y, X_with_support).fit()

print('Koeffizient Log_Homicides ohne SocialSupport:', model_simple.params['Log_Homicides'])
print('Koeffizient Log_Homicides mit SocialSupport:', model_with_support.params['Log_Homicides'])


#### Interpretation

Wenn der Koeffizient für `Log_Homicides` im Modell ohne SocialSupport deutlich größer ist als im Modell mit SocialSupport, dann wirkt `SocialSupport` als **Konfundierer**: Die soziale Unterstützung hängt negativ mit der Mordrate zusammen und positiv mit dem Sicherheitsgefühl. Ohne Kontrolle für SocialSupport überschätzt das Modell den (negativen) Zusammenhang zwischen Mordrate und Sicherheitsgefühl.

## 6. Panelmodell (Fixed Effects)

In der Vorlesung wurde die **Panelanalyse** als Erweiterung der Regression vorgestellt. Paneldaten enthalten wiederholte Messungen (z. B. Länder über Jahre). Ein Fixed-Effects-Modell kontrolliert für unbeobachtete, zeitinvariante Unterschiede zwischen Ländern. Wir verwenden den Zeitreihendatensatz und betrachten die gleichen Variablen wie zuvor.

Zuerst bereiten wir den Datensatz vor: Wir filtern die benötigten Kennzahlen, bilden die gleiche Transformation der Mordrate und stellen die Daten in ein Panelformat mit Index (Land, Jahr).

In [ ]:

# Relevante Kennzahlen für das Panel
measures_panel = ['Feeling safe at night', 'Homicides', 'Social support']
df_ts_filt = df_ts[df_ts['measure'].isin(measures_panel)].copy()
df_ts_filt['value'] = pd.to_numeric(df_ts_filt['value'], errors='coerce')
# Pivotieren: eine Zeile pro Land/Jahr
df_ts_panel = df_ts_filt.pivot_table(index=['reference_area', 'year'], columns='measure', values='value', aggfunc='mean')
df_ts_panel = df_ts_panel.dropna()
df_ts_panel = df_ts_panel.rename(columns={
    'Feeling safe at night': 'FeelingSafe',
    'Homicides': 'Homicides',
    'Social support': 'SocialSupport'
})
df_ts_panel['Log_Homicides'] = np.log(df_ts_panel['Homicides'] + 1e-6)

# Panelindex setzen
panel_df = df_ts_panel.set_index(['reference_area', 'year'])

# Panelmodell schätzen (Fixed Effects)
panel_model = PanelOLS.from_formula('FeelingSafe ~ 1 + Log_Homicides + SocialSupport + EntityEffects', data=panel_df).fit()
print(panel_model.summary)


#### Interpretation des Panelmodells

- Das **Fixed-Effects-Modell** absorbiert alle zeitinvarianten länderspezifischen Einflüsse (z. B. Kultur, grundlegendes Sicherheitsniveau).
- Die Koeffizienten von `Log_Homicides` und `SocialSupport` messen die Veränderung des Sicherheitsgefühls **innerhalb** eines Landes über die Zeit, wenn sich die Mordrate bzw. soziale Unterstützung ändern.
- Ein signifikanter negativer Koeffizient für `Log_Homicides` bedeutet, dass eine Zunahme der Mordrate in einem Land (über die Zeit) mit einem Rückgang des Sicherheitsgefühls einhergeht, selbst wenn länderspezifische Unterschiede berücksichtigt werden. Ein positiver Koeffizient für `SocialSupport` deutet darauf hin, dass mehr soziale Unterstützung im Land über die Zeit zu einem höheren Sicherheitsgefühl führt.

Die Panelanalyse ist besonders wertvoll, wenn unbeobachtete Heterogenität zwischen den Einheiten die Schätzungen verzerren könnte.

## 7. Fazit

In diesem Notebook haben wir zentrale Konzepte aus Vorlesung 11 angewendet:

- **Multiple Regression** ermöglicht die Abschätzung von **partiellen Effekten** und kontrolliert für andere Einflussfaktoren.
- **Dummy-Codierung** wird genutzt, um kategoriale Variablen in lineare Modelle einzubringen und Interpretationen zu erleichtern.
- **Multikollinearität** kann über den **VIF** diagnostiziert werden; hohe Werte warnen vor instabilen Koeffizienten.
- **Konfundierung** zeigt sich, wenn der Effekt einer Variablen sich ändert, sobald weitere Variablen ins Modell aufgenommen werden.
- **Panelmodelle** (Fixed Effects) erlauben es, unbeobachtete, zeitinvariante Unterschiede zu kontrollieren und so robustere Effekte in längsschnittlichen Daten zu schätzen.

Diese Analysen bauen auf den Erkenntnissen der vorherigen Notebooks auf und folgen der logischen Lernreise der Vorlesungen.